# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")

do_cat = False                        # Recreate catalog file from vasca_pipe output region
do_cross_match = False                  # Does source matching with CDS
do_src_prep = True                  # Gets additional info for sources listes in srcs_ids list below.
do_variable_addition = False        # Adds variables of matched sources to tt_sources
do_LombScargle = False


srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,253670
            ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,645215 ,645730
            ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206 ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE
srcs_ids = [4993,14504,52994,120265,221349,236839,253214,381336,525310,533185,639720,666224,673175,680119,1070938] # # WD ALL_10-800-LOOSE BB SED# Additional candidates 139480,204419,532554,659787,79477,

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname) 
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad", overwrite=False)
    rc.cross_match_cds(overwrite=True)

    #Crossmatch with GFCAT
    tt_cat = Table.read("./resources/GFCAT/hlsp_gfcat_galex_imaging_object-cat_fuv-nuv_v1_table.ecsv")
    rc.add_table(tt_cat,"tt_gfcat")
    rc.cross_match(dist_max=1 * uu.arcsec, dist_s2n_max=None, cat_table_name="tt_gfcat",cat_id_name="gfcat_src_id",cat_name="gfcat", src_table_name="tt_sources")
    
    #rc.cross_match_cds(query_table="I/358/varisum",vizier_columns=["*", "NG", "dGmag"],)

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0
    rc.tt_gaiadr3["Plx_dist"] = rc.tt_gaiadr3["Plx"].quantity.to(uu.parsec, equivalencies=uu.parallax())

    # Add normalized excess
    rc.tt_sources["flux_ne"]=np.sqrt(rc.tt_sources["flux_nxv"])

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

if do_LombScargle:
    rc.set_LombScargle(nbins_min=20)

#Write out all changes to region
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        print(f"*********** Creating Source Table ID {src_id} *****************")
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-11-15 09:06:05.598 | DEBUG    | vasca.tables:load_from_fits:330 - Loading file with name './vasca_pipeline/ALL_10-800_LOOSE/region_ALL_10-800_LOOSE_cat.fits'
2023-11-15 09:06:05.605 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_fields'
2023-11-15 09:06:05.616 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_filters'
2023-11-15 09:06:05.621 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_visits'
2023-11-15 09:06:05.628 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_sources'
2023-11-15 09:06:05.650 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_detections'
2023-11-15 09:06:05.682 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_detections'
2023-11-15 09:06:05.752 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_sources'
2023-11-15 09:06:05.767 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_src_id_map'
2023-11-15 09:06:05.774 | DEBUG   

*********** Creating Source Table ID 4993 *****************


2023-11-15 09:06:16.478 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-15 09:06:16.484 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.485 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.485 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.486 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.486 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.487 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.487 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.487 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:16.487 | WARNING  | vasca.source:add_vizier_SED:99 - Ski

*********** Creating Source Table ID 14504 *****************


2023-11-15 09:06:30.098 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-15 09:06:30.155 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.157 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.157 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.157 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.158 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.158 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.159 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.159 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:06:30.159 | WARNING  | vasca.source:add_vizier_SED:99 - Ski

...

*********** Creating Source Table ID 680119 *****************


2023-11-15 09:08:11.466 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-15 09:08:11.468 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:11.475 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:11.475 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:11.476 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:11.476 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:11.476 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:11.502 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_gphoton_lc'
2023-11-15 09:08:11.509 | WARNING  | vasca.source:add_spectrum:276 - No spectrum found
2023-11-15 09:08:11.509 | INFO     | vasca.tables:write_to_fits:224 - Writing file with name

*********** Creating Source Table ID 1070938 *****************


2023-11-15 09:08:19.046 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-15 09:08:19.063 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:19.063 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:19.064 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-15 09:08:19.080 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_gphoton_lc'
2023-11-15 09:08:19.084 | WARNING  | vasca.source:add_spectrum:276 - No spectrum found
2023-11-15 09:08:19.084 | INFO     | vasca.tables:write_to_fits:224 - Writing file with name './vasca_pipeline/ALL_10-800_LOOSE/sources/src_1070938.fits'
2023-11-15 09:08:19.086 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sources'
2023-11-15 09:08:19.113 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_detections'
2023-11-15 09:08:19.123 | DEBUG    | vasca.tables:wri

Done
